# Simple Grover's Search with the Azure Quantum Service

This notebook demonstrates how to use Q# and the Azure Quantum service together to search for data with Grover's algorithm, an example of the quantum development technique known as amplitude amplification. By applying a sequence of reflections, this sample prepares a register of qubits in a state marked by a given quantum operation known as an oracle. The oracle used in this sample checks if its input matches a given integer, so that the computational basis state corresponding to that index is prepared with high probability.

## Grover's diffusion operator
First, define the operations that will be used to create the Grover diffusion operator. This can take 30 seconds to run the first time, as the Q# environment is initializing.

In [5]:
open Microsoft.Quantum.Arrays;

operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(H, inputQubits);
}

operation PrepareAllOnes(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(X, inputQubits);
}

operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
    Controlled Z(Most(inputQubits), Tail(inputQubits));
}

PrepareUniform, PrepareAllOnes, ReflectAboutAllOnes

Next, define the diffusion operator. This reflects the input qubits about the uniform superposition state.

In [6]:
operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
    within {
        // Transform the uniform superposition to all-zero.
        Adjoint PrepareUniform(inputQubits);
        // Transform the all-zero state to all-ones
        PrepareAllOnes(inputQubits);
    } apply {
        // Now that we've transformed the uniform superposition to the
        // all-ones state, reflect about the all-ones state, then let
        // the within/apply block transform us back.
        ReflectAboutAllOnes(inputQubits);
    }
}

ReflectAboutUniform

## Grover's operation
Now we write a Q# operation for Grover's search algorithm. Start by defining an function that will return the number of Grover iterations needed to find a single marked item, given the number of qubits in a register. 

In [7]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;

function NIterations(nQubits : Int) : Int {
    let nItems = 1 <<< nQubits; // 2^numQubits
    // compute number of iterations:
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let nIterations = Round(0.25 * PI() / angle - 0.5);
    return nIterations;
}

NIterations

Next, define an operation to reflect the input register about the basis state marked by a given index. This operation defines the input we are trying to find in the main search.

In [8]:
operation ReflectAboutMarked(idxMarked : Int, inputQubits : Qubit[]) : Unit {
    use outputQubit = Qubit();
    within {
        // We initialize the outputQubit to (|0⟩ - |1⟩) / √2,
        // so that toggling it results in a (-1) phase.
        X(outputQubit);
        H(outputQubit);
    } apply {
        // Flip the outputQubit for marked states.
        // Here, we get the state given by the index idxMarked.
        (ControlledOnInt(idxMarked, X))(inputQubits, outputQubit);
    }
}

ReflectAboutMarked

Finally, we apply Grover's algorithm to search all possible inputs to an operation to find a particular marked state. To do this, we create an operation that takes the number of qubits to be allocated and the index of the marked item to be found. The output will be the computational basis state found in the final measurement. The output should be a little-endian representation of the marked index.

In [9]:
open Microsoft.Quantum.Measurement;

operation SearchForMarkedInput(nQubits : Int, idxMarked : Int) : Result[] {
    use qubits = Qubit[nQubits];
    // Initialize a uniform superposition over all possible inputs.
    PrepareUniform(qubits);
    // The search itself consists of repeatedly reflecting about the
    // marked state and our start state, which we can write out in Q#
    // as a for loop.
    for _ in 0..NIterations(nQubits) - 1 {
        ReflectAboutMarked(idxMarked, qubits);
        ReflectAboutUniform(qubits);
    }
    // Measure and return the answer.
    return ForEach(MResetZ, qubits);
}

SearchForMarkedInput

We can run a local simulation of this operation using the [`%simulate`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/simulate) command.

In [10]:
%simulate SearchForMarkedInput nQubits=3 idxMarked=6

Zero, One, One

## Running Grover's algorithm in Azure Quantum
To run the operation on the Azure Quantum service, you must first connect to your workspace by running [`%azure.connect`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/azure.connect) with the workspace's resource ID and location.

In [ ]:
%azure.connect "" location=""

Then use [`%azure.target`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/azure.target) to select the target you'd like to use for job submission. This may take a minute or so. Note that this specific sample is not supported for Rigetti targets.

In [2]:
%azure.target ionq.simulator

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.simulator,Available,1


To submit the job, use [`%azure.submit`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/azure.submit) with the Q# operation name and any required parameters. This will return immediately after the job is created. Alternatively, you can use [`%azure.execute`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/azure.execute)., which will submit the job and wait for it to complete. 

In [ ]:
%azure.submit SearchForMarkedInput nQubits=3 idxMarked=6

You can check the status of the job using [`%azure.status`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/azure.status). Calling this command without any input will check the status of the job most recently submitted in this session. You can also pass a job ID to check the status of a different job.

In [ ]:
%azure.status

Once the job is complete, you can view the output using the [`%azure.output`](https://docs.microsoft.com/qsharp/api/iqsharp-magic/azure.output) command. This magic command will also accept a job ID to check the output of a previously submitted job.

In [13]:
%azure.output

Result,Frequency,Histogram
"[0,0,0]",0.0078125,
"[1,0,0]",0.0078125,
"[0,1,0]",0.0078125,
"[1,1,0]",0.0078125,
"[0,0,1]",0.0078125,
"[1,0,1]",0.0078125,
"[0,1,1]",0.9453125,
"[1,1,1]",0.0078125,
